In [1]:

import geopandas as gpd
from shapely.ops import nearest_points
from shapely.geometry import Point
import numpy as np
import pandas as pd


In [11]:
highways_gdf = gpd.read_file(r"D:\CS\GitHub\Nuclear-Site-Selection\Highways and Roadways data\Highways\National_Highways.shp")
dams_gdf = gpd.read_file(r'D:\CS\GitHub\Nuclear-Site-Selection\dam\Dam.shp')
dams_csv = pd.read_csv(r"D:\CS\GitHub\Nuclear-Site-Selection\dam\dam.csv")

In [12]:
# Ensure both GeoDataFrames use the same CRS
dams_gdf = dams_gdf.to_crs(highways_gdf.crs)

In [13]:
# Function to find the nearest geometry, distance, and highway name using an incremental buffer approach
def nearest_with_buffer(row, other_gdf, point_column='geometry', initial_buffer=10000, buffer_increment=10000):
    buffer_distance = initial_buffer
    point = row[point_column]
    
    while True:
        buffer = point.buffer(buffer_distance)
        potential_highways = other_gdf[other_gdf.intersects(buffer)]
        
        if not potential_highways.empty:
            # Calculate the distances to all intersecting linestrings and find the minimum
            distances = potential_highways.geometry.apply(lambda geom: point.distance(geom))
            min_idx = distances.idxmin()
            nearest_geom = potential_highways.loc[min_idx, 'geometry']
            highway_name = potential_highways.loc[min_idx, 'name']  # Ensure 'ref' is the column name for highway names
            distance = distances[min_idx]
            return nearest_geom, distance, highway_name
        
        # Increase buffer distance and repeat
        buffer_distance += buffer_increment

In [14]:
# Calculate the nearest highway and distance for each dam
nearest_highways = dams_gdf.apply(lambda row: nearest_with_buffer(row, highways_gdf), axis=1)


In [15]:
# Initialize lists to store results
nearest_highway_geometries = []
nearest_highway_distances = []
nearest_highway_names = []

# Populate the lists with the results
for item in nearest_highways:
    nearest_highway_geometries.append(item[0])
    nearest_highway_distances.append(item[1] if item[1] is not None else np.nan)
    nearest_highway_names.append(item[2])

# Update the GeoDataFrame with the new columns
dams_gdf['nearest_highway_geometry'] = nearest_highway_geometries
dams_gdf['nearest_highway_distance'] = nearest_highway_distances
dams_gdf['nearest_highway'] = nearest_highway_names

In [17]:
# Update the GeoDataFrame with the new columns
dams_csv['nearest_highway_geometry'] = nearest_highway_geometries
dams_csv['nearest_highway_distance'] = nearest_highway_distances
dams_csv['nearest_highway'] = nearest_highway_names

In [20]:
dams_csv = dams_csv.drop(columns='Unnamed: 0')

In [22]:
dams_csv.to_csv(r"D:\CS\GitHub\Nuclear-Site-Selection\dam\dam.csv")